Download Model

In [ ]:
%%bash
huggingface-cli download mys/ggml_llava-v1.5-7b \
                mmproj-model-f16.gguf  \
                ggml-model-q4_k.gguf  \
                --local-dir ~/gai/models/ggml_llava-v1.5-7b \
                --local-dir-use-symlinks False


1. Generating

In [1]:
from gai.gen.ttt.LlamaCpp_TTT import LlamaCpp_TTT
config = {
            "type": "itt",
            "model_name": "llamacpp-llava",
            "engine": "LlamaCpp_ITT",
            "model_filepath": "models/ggml_llava-v1.5-7b/ggml-model-q4_k.gguf",
            "clip_model_path": "models/ggml_llava-v1.5-7b/mmproj-model-f16.gguf",
            "max_seq_len": 8192,
            "prompt_format": "llama3",
            "stop": ["<|eot_id|>"],
            "hyperparameters": {
                "max_tokens": 100,
                "temperature": 1.31,
                "top_k": 49,
                "top_p": 0.14
            }
        }   
gen = LlamaCpp_TTT(config)
gen.load()
import base64
encoded_string = ""
with open("./buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

response = gen.create(messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_string}",
                    },
                },
            ],
        }
    ],
    max_tokens=4000,
    stream=False)
gen.unload()
response

2024-06-12 21:49:24 INFO gai.gen.ttt.LlamaCpp_TTT:exllama_engine.load: Loading model from /home/roylai/gai/models/ggml_llava-v1.5-7b/ggml-model-q4_k.gguf


ChatCompletion(id='chatcmpl-77fc73c6-c88b-4242-8825-6d3e34272140', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='\n\n[0.12, 0.74, 0.35, 0.86]\n[0.12, 0.74, 0.35, 0.86]\n[0.12, 0.74, 0.35, 0.86]\n[0.12, 0.74, 0.35, 0.86', role='assistant', function_call=None, tool_calls=None))], created=1718200194, model='llamacpp-llava', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=102, prompt_tokens=20194, total_tokens=20296))

2. Streaming

In [2]:
from gai.gen.ttt.LlamaCpp_TTT import LlamaCpp_TTT
config = {
            "type": "ttt",
            "model_name": "Llama3-LlamaCpp",
            "engine": "LlamaCpp_TTT",
            "model_filepath": "models/LLaMA3-iterative-DPO-final-GGUF/LLaMA3-iterative-DPO-final-Q4_K_M.gguf",
            "max_seq_len": 8192,
            "prompt_format": "llama3",
            "stop": ["<|eot_id|>"],
            "hyperparameters": {
                "max_tokens": 100,
                "temperature": 1.31,
                "top_k": 49,
                "top_p": 0.14
            }
        }   
gen = LlamaCpp_TTT(config)
gen.load()
response = gen.create(messages=[
    {'role':'system','content':'You are a helpful assistant that can generate short stories. You can generate a short story based on a prompt.'},
    {'role':'user','content':'Tell me a one paragraph short story.'},
    {'role':'assistant','content':''}],
    max_tokens=500,
    stream=True,
    seed=1234)
for chunk in response:
    chunk=chunk.choices[0].delta.content
    if chunk:
        print(chunk, end='', flush=True)
gen.unload()

2024-06-12 20:05:03 INFO gai.gen.ttt.LlamaCpp_TTT:exllama_engine.load: Loading model from /home/roylai/gai/models/LLaMA3-iterative-DPO-final-GGUF/LLaMA3-iterative-DPO-final-Q4_K_M.gguf


In the quaint village of Willowbrook, nestled between rolling hills and a sparkling river, there lived an old woman named Elsie. She was known for her kind heart and wise words, often sharing stories with the children at the local school. One sunny afternoon, as she tended to her garden, Elsie discovered a tiny, lost bird perched on a flower petal. With gentle care, she coaxed it into her hand, naming him Whispers. Over time, their bond grew strong; Whispers would visit Elsie daily, bringing joy and comfort to the villagers who marveled at this extraordinary friendship between an old woman and a small bird. And so, in Willowbrook, love transcended boundaries, proving that even the smallest creatures can touch our hearts and remind us of life's simple beauty.

---
## JSON Mode

In [3]:
from gai.gen.ttt.LlamaCpp_TTT import LlamaCpp_TTT
config = {
            "type": "ttt",
            "model_name": "Llama3-LlamaCpp",
            "engine": "LlamaCpp_TTT",
            "model_filepath": "models/LLaMA3-iterative-DPO-final-GGUF/LLaMA3-iterative-DPO-final-Q4_K_M.gguf",
            "max_seq_len": 8192,
            "prompt_format": "llama3",
            "stop": ["<|eot_id|>"],
            "hyperparameters": {
                "max_tokens": 100,
                "temperature": 1.31,
                "top_k": 49,
                "top_p": 0.14
            }
        }   
gen = LlamaCpp_TTT(config)
gen.load()

# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""
response = gen.create(messages=[{'role':'user','content':text},{'role':'assistant','content':''}], 
    schema=Book.schema()
    )
gen.unload()
response

2024-06-12 20:05:55 INFO gai.gen.ttt.LlamaCpp_TTT:exllama_engine.load: Loading model from /home/roylai/gai/models/LLaMA3-iterative-DPO-final-GGUF/LLaMA3-iterative-DPO-final-Q4_K_M.gguf
from_string grammar:
author-kv ::= ["] [a] [u] [t] [h] [o] [r] ["] space [:] space string 
space ::= space_43 
string ::= ["] string_44 ["] space 
char ::= [^"\] | [\] char_4 
char_4 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
integer ::= integer_6 space 
integer_6 ::= integer_7 integral-part 
integer_7 ::= [-] | 
integral-part ::= [0-9] | [1-9] integral-part_38 
integral-part_9 ::= [0-9] integral-part_37 
integral-part_10 ::= [0-9] integral-part_36 
integral-part_11 ::= [0-9] integral-part_35 
integral-part_12 ::= [0-9] integral-part_34 
integral-part_13 ::= [0-9] integral-part_33 
integral-part_14 ::= [0-9] integral-part_32 
integral-part_15 ::= [0-9] integral-part_31 
integral-part_16 ::= [0-9] integral-part_30 
integral-part_17 ::= [0-9] integral-part_29 
integral-part_18 ::

ChatCompletion(id='chatcmpl-40d25c24-51f7-4d80-a1c6-289c1457da89', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='{"title": "Foundation", "summary": "Foundation is a seminal science fiction novel written by American author Isaac Asimov. It was first published in 1951 as part of his Foundation Trilogy, which later expanded into the larger Foundation series. The book comprises five interconnected short stories that collectively narrate the early history of the Foundation, an institution established by psychohistorian Hari Seldon to safeguard and preserve the most valuable aspects of galactic civilization after the anticipated fall of the Galactic Empire.\n\nThe', role='assistant', function_call=None, tool_calls=None))], created=1718193994, model='Llama3-LlamaCpp', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=101, prompt_tokens=102, total_tokens=203))

---
## Function Calling

In [6]:
from gai.gen.ttt.LlamaCpp_TTT import LlamaCpp_TTT
config = {
            "type": "ttt",
            "model_name": "Llama3-LlamaCpp",
            "engine": "LlamaCpp_TTT",
            "model_filepath": "models/LLaMA3-iterative-DPO-final-GGUF/LLaMA3-iterative-DPO-final-Q4_K_M.gguf",
            "max_seq_len": 8192,
            "prompt_format": "llama3",
            "stop": ["<|eot_id|>"],
            "hyperparameters": {
                "max_tokens": 100,
                "temperature": 1.31,
                "top_k": 49,
                "top_p": 0.14
            }
        }   
gen = LlamaCpp_TTT(config)
gen.load()

from gai.common.notebook import highlight
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
highlight("Model decided to use tool: ")
user_prompt = "What time is it in Singapore right now?"
response = gen.create(
    messages=[
        {'role':'user','content':user_prompt},
        {'role':'assistant','content':''}],
    tools=tools,
    stream=False,
    max_tokens=200)
print(response)

highlight("Model decided not to use tool: ")
user_prompt = "Tell me a one paragraph story."
response = gen.create(
    messages=[
        {'role':'user','content':user_prompt},
        {'role':'assistant','content':''}],
    tools=tools,
    stream=False,
    max_tokens=200)
print(response)


2024-06-12 20:09:04 INFO gai.gen.ttt.LlamaCpp_TTT:exllama_engine.load: Loading model from /home/roylai/gai/models/LLaMA3-iterative-DPO-final-GGUF/LLaMA3-iterative-DPO-final-Q4_K_M.gguf


from_string grammar:
array ::= [[] space array_6 []] space 
space ::= space_94 
array_2 ::= value array_5 
value ::= object | array | string | number | boolean | null 
array_4 ::= [,] space value 
array_5 ::= array_4 array_5 | 
array_6 ::= array_2 | 
boolean ::= boolean_8 space 
boolean_8 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] 
char ::= [^"\] | [\] char_10 
char_10 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
decimal-part ::= [0-9] decimal-part_41 
decimal-part_12 ::= [0-9] decimal-part_40 
decimal-part_13 ::= [0-9] decimal-part_39 
decimal-part_14 ::= [0-9] decimal-part_38 
decimal-part_15 ::= [0-9] decimal-part_37 
decimal-part_16 ::= [0-9] decimal-part_36 
decimal-part_17 ::= [0-9] decimal-part_35 
decimal-part_18 ::= [0-9] decimal-part_34 
decimal-part_19 ::= [0-9] decimal-part_33 
decimal-part_20 ::= [0-9] decimal-part_32 
decimal-part_21 ::= [0-9] decimal-part_31 
decimal-part_22 ::= [0-9] decimal-part_30 
decimal-part_23 ::= [0-9] decimal-part_29 
dec

ChatCompletion(id='chatcmpl-f3eabcbe-a353-42cd-817f-7f5e1bc7c187', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{ "function": { "name": "google", "arguments": { "search_query": "current time in Singapore" } } }', role='assistant', function_call=None, tool_calls=None))], created=1718194176, model='Llama3-LlamaCpp', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=345, total_tokens=374))


from_string grammar:
array ::= [[] space array_6 []] space 
space ::= space_94 
array_2 ::= value array_5 
value ::= object | array | string | number | boolean | null 
array_4 ::= [,] space value 
array_5 ::= array_4 array_5 | 
array_6 ::= array_2 | 
boolean ::= boolean_8 space 
boolean_8 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] 
char ::= [^"\] | [\] char_10 
char_10 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
decimal-part ::= [0-9] decimal-part_41 
decimal-part_12 ::= [0-9] decimal-part_40 
decimal-part_13 ::= [0-9] decimal-part_39 
decimal-part_14 ::= [0-9] decimal-part_38 
decimal-part_15 ::= [0-9] decimal-part_37 
decimal-part_16 ::= [0-9] decimal-part_36 
decimal-part_17 ::= [0-9] decimal-part_35 
decimal-part_18 ::= [0-9] decimal-part_34 
decimal-part_19 ::= [0-9] decimal-part_33 
decimal-part_20 ::= [0-9] decimal-part_32 
decimal-part_21 ::= [0-9] decimal-part_31 
decimal-part_22 ::= [0-9] decimal-part_30 
decimal-part_23 ::= [0-9] decimal-part_29 
dec

ChatCompletion(id='chatcmpl-8e4a4cb0-d9ef-41ad-a2eb-0ee0fa121f6f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{ "function": { "name": "google", "arguments": { "search_query": "one paragraph story" } } }', role='assistant', function_call=None, tool_calls=None))], created=1718194185, model='Llama3-LlamaCpp', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=343, total_tokens=371))
